In [ ]:
!pip install retrying

In [ ]:
import requests
import json

def scrape_data_pendidikan(tingkat):
    """
    Fungsi untuk melakukan scraping data pendidikan dari API SSCASN
    dan hanya memproses data yang tidak kosong.

    Args:
        tingkat: Tingkat pendidikan (integer).

    Returns:
        List of dictionaries, where each dictionary represents data pendidikan,
        or None jika terjadi error.
    """

    url = f'https://api-sscasn.bkn.go.id/2024/referensi/pendidikan?tingkat={tingkat}&nama=null&limit=2500'
    headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-encoding': 'gzip, deflate, br, zstd',
        'accept-language': 'en-US,en;q=0.9,id;q=0.8',
        'connection': 'keep-alive',
        'host': 'api-sscasn.bkn.go.id',
        'origin': 'https://sscasn.bkn.go.id',
        'referer': 'https://sscasn.bkn.go.id/',
        'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
        'Cookie': '8031e2dda37b1552a45b6fe38d7ed11d=65565f4273ba46ec68d6679358167f93; BIGipServerpool_prod_sscasn2024_kube=3473433610.47873.0000; 8031e2dda37b1552a45b6fe38d7ed11d=65565f4273ba46ec68d6679358167f93; BIGipServerpool_prod_sscasn2024_kube=3473433610.47873.0000'
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        data = response.json()

        # Periksa apakah data kosong
        if data['data']:
            return data['data']  # Kembalikan data jika tidak kosong
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

if __name__ == "__main__":
    all_data_pendidikan = []
    for tingkat in range(1, 51): #data tingkat pendidikan maksimal divalue 50
        data_pendidikan = scrape_data_pendidikan(tingkat)
        if data_pendidikan and 'data' in data_pendidikan:  # Pastikan data_pendidikan tidak None
            for item in data_pendidikan['data']:
              result = {
                  'kode_pend' : item['kode_pend'],
                  'nama_pend' : item['nama_pend']
              }
              all_data_pendidikan.append(result)


        #  Simpan data ke file JSON
with open('data_pendidikan.json', 'w') as f:
  json.dump(all_data_pendidikan, f, indent=4)
  print("Data berhasil disimpan ke data_pendidikan.json")

In [ ]:
import requests
import json
import time
from retrying import retry

@retry(stop_max_attempt_number=3, wait_fixed=2000)
def get_data(kode_pendidikan, offset=0, limit=10):
    """
    Fungsi untuk mengambil data formasi dari API SSCASN
    berdasarkan kode pendidikan dengan retry logic dan pagination.
    """
    try:
        url = 'https://api-sscasn.bkn.go.id/2024/portal/spf'
        params = {
            'kode_ref_pend': kode_pendidikan,
            'pengadaan_kd': '2',
            'offset': offset,
            'limit': limit,
            'nama': '',
            'instansi': ''
        }
        headers = {
            'accept': 'application/json, text/plain, */*',
            'accept-language': 'en-US,en;q=0.9,id;q=0.8',
            'origin': 'https://sscasn.bkn.go.id',
            'referer': 'https://sscasn.bkn.go.id/',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }

        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        response_json = response.json()

        # print(f"\nResponse untuk kode {kode_pendidikan} (offset: {offset}, limit: {limit}):")
        # print(json.dumps(response_json, indent=2))

        return response_json

    except requests.exceptions.RequestException as e:
        print(f"Error mengambil data untuk kode {kode_pendidikan}: {e}")
        raise

def get_all_data(kode_pendidikan):
    """
    Fungsi untuk mengambil semua data dengan pagination.
    """
    offset = 0
    limit = 10  # Sesuaikan dengan limit API
    all_data = []

    while True:
        response = get_data(kode_pendidikan, offset, limit)
        data = response.get('data', [])
        if data:
            all_data.extend(data['data'])
        else:
            break

        # Periksa apakah ada data lagi
        if len(data['data']) < limit:
            break

        offset += limit
        time.sleep(1)  # Menambahkan delay untuk menghindari rate limiting

    return all_data

def get_kode_pend(file_name):
    """
    Fungsi untuk mengambil semua 'kode_pend' dari file JSON.
    """
    try:
        with open(file_name, 'r') as f:
            data = json.load(f)
        return [item['kode_pend'] for item in data]
    except Exception as e:
        print(f"Error membaca file: {e}")
        return None

if __name__ == "__main__":
    file_name = 'data_pendidikan.json'
    kode_pend_result = get_kode_pend(file_name)

    if kode_pend_result:
        all_data = []
        for i, kode in enumerate(kode_pend_result):
            print(f"\nMengambil data untuk kode pendidikan: {kode}")
            try:
                data = get_all_data(kode)
                if data:
                    all_data.extend(data)
                    print(f"Berhasil mendapatkan {len(data)} data untuk kode {kode}")
                else:
                    print(f"Tidak ada data untuk kode pendidikan {kode}")
            except requests.exceptions.RequestException:
                print(f"Gagal mengambil data untuk kode {kode} setelah beberapa kali mencoba.")

        if all_data:
            with open('hasil_formasi.json', 'w') as f:
                json.dump(all_data, f, indent=2)
            print(f"\nTotal data yang berhasil dikumpulkan: {len(all_data)}")
        else:
            print("\nTidak ada data yang berhasil dikumpulkan")